# Rheostat Modeling Attempt 3

In [1]:
#Using Bioscrape: Basic Imports

#A Model is a CRN with some bells and whistles
from bioscrape.types import Model

#py_simulate_model is a helper function that takes care of may details for you
from bioscrape.simulator import py_simulate_model

#For arrays and plotting
import numpy as np
import pylab as plt

# Import good plotting packages 
import bokeh.io
import bokeh.plotting
#import bokeh_catplot


bokeh.io.output_notebook()

Loading BokehJS ...

Now, we will introduce a half-way point (with fructose-1,6-phosphate) and attempt to model the three different ways that enzyme substrate binding can occur. The half-way point is necessary so we can get the free phosphate concentrations that come from the Non-ATP generating pathway.

## Try modeling the 1 step model with hillpositive

$$\text{Fuel + Substrate + Enzyme} \leftrightarrow^1 \text{Fuel:Substrate:Enzyme} \rightarrow^2 \text{Waste:Product:Enzyme} \leftrightarrow^3 \text{Waste + Product + Enzyme}$$

**Non-ATP Generating Pathway** <br>
$$\text{2 ATP + Glucose + Enzyme1} \leftrightarrow^1 \text{2ATP:Glucose:Enzyme1} \rightarrow^2 \text{2ADP2Pi:F16P:Enzyme1} \leftrightarrow^3 \text{2 ADP + 2 Pi + F16P + Enzyme1}$$ <br>

$$\text{F16P + 2 ADP + 2 Pi + Enzyme2} \leftrightarrow^4 \text{2ADP2Pi:F16P:Enzyme2} \rightarrow^5 \text{2 ATP:Isobutanol:Enzyme2} \leftrightarrow^6 \text{2 ATP + Isobutanol + Enzyme2}$$ <br>

**ATP Generating Pathway** <br>
$$\text{2ATP + Glucose + Enzyme3} \leftrightarrow^1 \text{2ATP:Glucose:Enzyme3} \rightarrow^2 \text{2ADP2Pi:F16P:Enzyme3} \leftrightarrow^3 \text{2 ADP + 2 Pi + F16P + Enzyme3}$$ <br>

$$\text{2 ATP + 2 Pi + ADP + F16P + Enzyme4} \leftrightarrow^4 \text{2ATP2PiADP:F16P:Enzyme4} \rightarrow^5 \text{3ATP:Isobutanol:Enzyme4} \leftrightarrow^6 \text{3 ATP + Isobutanol + Enzyme4}$$ <br>

**ATP Degradation** <br>
$$\text{ATP} \rightarrow \text{ADP + Pi}$$

In [18]:
species = ['atp', 'glucose', 'e1', '2atp:glucose:e1', '2adp2pi:f16p:e1', 'adp', 'pi', 'f16p',
           'e2', '2adp2pi:f16p:e2','2atp:isobutanol:e2', 
           '2adp2pi:f16p:e3', 'e3' , '2atp:glucose:e3',
           'e4', '2atp2piadp:f16p:e4', '3atp:isobutanol:e4', 'isobutanol']


parameters = [("kf", 10), ("kr",0.1),
            ("delta", 10), ("n", 1),
             ('K_hill', 10)]


# NON ATP GENERATING
# reaction 1
rxn1_non_f = (['atp', 'atp', 'glucose', 'enzyme1'], ['2atp:glucose:e1'],
           'hillpositive',  {"s1":"glucose", "k":"kf", "K":"K_hill", "n":'n'})
                            
rxn1_non_r = ( ['2atp:glucose:e1'],['atp', 'atp', 'glucose', 'e1'], 
            'hillpositive', {"s1":"2atp:glucose:e1", "k":"kr", "K":"K_hill", "n":'n'})

# reaction 2
rxn2_non_f = ( ['2atp:glucose:e1'],['2adp2pi:f16p:e1'], 
           'hillpositive', {"s1":"2atp:glucose:e1", "k":"kf", "K":"K_hill", "n":'n'})

# reaction 3
rxn3_non_f =  ( ['2adp2pi:f16p:e1'], ['adp', 'adp', 'pi', 'pi', 'f16p', 'e1'], 
           'hillpositive', {"s1":"2adp2pi:f16p:e1", "k":"kf", "K":"K_hill", "n":'n'})
 
rxn3_non_r =  (['adp', 'adp', 'pi', 'pi', 'f16p', 'e1'], ['2adp2pi:f16p:e1'],
           'hillpositive', {"s1":"f16p", "k":"kr", "K":"K_hill", "n":'n'})

# reaction 4
rxn4_non_f =  (['f16p', 'adp', 'adp', 'pi', 'pi', 'e2'], ['2adp2pi:f16p:e2'],
           'hillpositive', {"s1":"f16p", "k":"kf", "K":"K_hill", "n":'n'})

rxn4_non_r =  ( ['2adp2pi:f16p:e2'], ['f16p', 'adp', 'adp', 'pi', 'pi', 'e2'],
           'hillpositive', {"s1":"2adp2pi:f16p:e2", "k":"kr", "K":"K_hill", "n":'n'})

# reaction 5
rxn5_non_f =  (['2adp2pi:f16p:e2'], ['2atp:isobutanol:e2'],
           'hillpositive', {"s1":"2adp2pi:f16p:e2", "k":"kf", "K":"K_hill", "n":'n'})

# reaction 6
rxn6_non_f =  (['2atp:isobutanol:e2'], ['atp', 'atp', 'isobutanol', 'e2'],
           'hillpositive', {"s1":"2atp:isobutanol:e2", "k":"kf", "K":"K_hill", "n":'n'})

rxn6_non_r =  (['atp', 'atp', 'isobutanol', 'e2'], ['2atp:isobutanol:e2'],
           'hillpositive', {"s1":"f16p", "k":"kr", "K":"K_hill", "n":'n'})
    
# ATP GENERATING
# reaction 1
rxn1_atp_f = ( ['atp', 'atp', 'glucose', 'e3'], ['2atp:glucose:e3'], 
              'hillpositive', {"s1":"glucose", "k":"kf", "K":"K_hill", "n":'n'})

rxn1_atp_r = ( ['2atp:glucose:e3'], ['atp', 'atp', 'glucose', 'e3'], 
              'hillpositive', {"s1":"2atp:glucose:e3", "k":"kr", "K":"K_hill", "n":'n'})

# reaction 2
rxn2_atp_f = ( ['2atp:glucose:e3'],['2adp2pi:f16p:e3'], 
           'hillpositive', {"s1":"2atp:glucose:e3", "k":"kf", "K":"K_hill", "n":'n'})


# reaction 3
rxn3_atp_f = ( ['2adp2pi:f16p:e3'], ['adp', 'adp', 'pi', 'pi', 'f16p', 'e3'], 
              'hillpositive', {"s1":"2adp2pi:f16p:e3", "k":"kf", "K":"K_hill", "n":'n'})

rxn3_atp_r = ( ['adp', 'adp', 'pi', 'pi', 'f16p', 'e3'],['2adp2pi:f16p:e3'], 
              'hillpositive', {"s1":"f16p", "k":"kr", "K":"K_hill", "n":'n'})

# reaction 4
rxn4_atp_f = ( ['atp', 'atp', 'pi', 'pi', 'adp', 'f16p', 'e4'],['2atp2piadp:f16p:e4'], 
              'hillpositive', {"s1":"f16p", "k":"kf", "K":"K_hill", "n":'n'})

rxn4_atp_r = ( ['2atp2piadp:f16p:e4'],['atp', 'atp', 'pi', 'pi', 'adp', 'f16p', 'e4'], 
              'hillpositive', {"s1":"2atp2piadp:f16p:e4", "k":"kr", "K":"K_hill", "n":'n'})

# reaction 5
rxn5_atp_r = ( ['2atp2piadp:f16p:e4'],['3atp:isobutanol:e4'], 
              'hillpositive', {"s1":"2atp2piadp:f16p:e4", "k":"kf", "K":"K_hill", "n":'n'})

# reaction 6
rxn6_atp_f = ( ['3atp:isobutanol:e4'],['atp', 'atp', 'isobutanol', 'e4'], 
              'hillpositive', {"s1":"3atp:isobutanol:e4", "k":"kf", "K":"K_hill", "n":'n'})

rxn6_atp_r = ( ['atp', 'atp', 'isobutanol', 'e4'],['3atp:isobutanol:e4'], 
              'hillpositive', {"s1":"isobutanol", "k":"kr", "K":"K_hill", "n":'n'})

# ATP DEGRADATION
rxn_d_atp = (["atp"], ['adp', 'pi'], "massaction", {"k":"delta"})


reactions = [rxn1_non_f, rxn1_non_r, rxn2_non_f, rxn3_non_f, rxn3_non_r,
             rxn4_non_f, rxn4_non_r, rxn5_non_f, rxn6_non_f, rxn6_non_r,
             rxn1_atp_f, rxn1_atp_r, rxn2_atp_f, rxn3_atp_f, rxn3_atp_r,
             rxn4_atp_f, rxn4_atp_r, rxn5_atp_r, rxn6_atp_f, rxn6_atp_r,
            rxn_d_atp]

# Initial Conditions             
x0 = {
    "glucose":100,
    "atp":200, 
    'enzyme1': 200, 
    'enzyme2':200,
}

rules = []

M = Model(species = species, reactions = reactions, parameters = parameters, rules = rules, initial_condition_dict = x0)
timepoints = np.linspace(0,100,1000)
df_results = py_simulate_model(timepoints, Model = M, stochastic = False)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:107: UserWarning: The following species are uninitialized and their value has been defaulted to 0: e1, 2atp:glucose:e1, 2adp2pi:f16p:e1, adp, pi, f16p, e2, 2adp2pi:f16p:e2, 2atp:isobutanol:e2, 2adp2pi:f16p:e3, e3, 2atp:glucose:e3, e4, 2atp2piadp:f16p:e4, 3atp:isobutanol:e4, isobutanol, 


In [19]:
#from bioscrape.sbmlutil import import_sbml
#from bioscrape.simulator import py_simulate_model
# Attempt SBML
#M.write_sbml_model('rheostat3.xml')

In [20]:
#M_sbml = import_sbml('rheostat3.xml')
#df_results = py_simulate_model(timepoints, Model = M_sbml, stochastic = False)

In [21]:
# First plot glucose and isobutanol
p = bokeh.plotting.figure(width = 300, height = 250, 
                         x_axis_label = 'time',
                         y_axis_label = 'concentration')
p.line(timepoints, df_results['glucose'], color = 'green', legend_label = 'glucose')
p.line(timepoints, df_results['isobutanol'], color = 'orange', legend_label = 'isobutanol')
p.legend.location = 'center_right'

# Plot ATP and ADP, NADPH, NADP+
p2 = bokeh.plotting.figure(width = 300, height = 250,
                           x_axis_label = 'time',
                         y_axis_label = 'concentration')
p2.line(timepoints, df_results['atp'], color = 'red', legend_label = 'atp')
p2.line(timepoints, df_results['pi'], color = 'blue', legend_label = 'pi')
p2.line(timepoints, df_results['adp'], color = 'green', legend_label = 'adp')
p2.legend.location = 'center_right'

#colors = ['#66c2a5', '#fc8d62','#8da0cb','#e78ac3']
colors = ['#e41a1c','#377eb8','#4daf4a','#984ea3']
p3 = bokeh.plotting.figure(width = 300, height = 250,
                           x_axis_label = 'time',
                         y_axis_label = 'concentration')
p3.line(timepoints, df_results['enzyme1'], color = 'blue', legend_label = 'enzyme1')
p3.line(timepoints, df_results['enzyme2'], color = 'green', legend_label = 'enzyme2')

p3.legend.location = 'center_right'


# Show plots
bokeh.io.show(p)
bokeh.io.show(p2)
bokeh.io.show(p3)

In [6]:
df_results

,atp,glucose,e1,2atp:glucose:e1,2adp2pi:f16p:e1,adp,pi,f16p,e2,2adp2pi:f16p:e2,...,2adp2pi:f16p:e3,e3,2atp:glucose:e3,e4,2atp2piadp:f16p:e4,3atp:isobutanol:e4,isobutanol,enzyme1,enzyme2,time
0,200.000000,2.000000e+02,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,80.000000,20.0,0.0000
1,194.216131,1.980946e+02,0.001880,9.091163e-01,4.212003e-02,1.978568,1.978497,2.747751e-03,-7.147337e-05,6.938166e-05,...,4.212003e-02,-9.512364e-01,9.091163e-01,-0.000071,6.938166e-05,0.000001,-6.462703e-07,79.046883,20.0,0.1001
2,188.492816,1.961916e+02,0.012126,1.743377e+00,1.502895e-01,3.927462,3.926454,1.902028e-02,-1.016455e-03,9.675521e-04,...,1.502895e-01,-1.893667e+00,1.743377e+00,-0.001007,9.675521e-04,0.000039,-6.957296e-06,78.094207,20.0,0.2002
3,182.825435,1.942907e+02,0.035772,2.518003e+00,3.042466e-01,5.862797,5.858249,5.571550e-02,-4.576588e-03,4.272533e-03,...,3.042466e-01,-2.822250e+00,2.518003e+00,-0.004534,4.272533e-03,0.000261,-1.503805e-05,77.141978,20.0,0.3003
4,177.208886,1.923916e+02,0.075941,3.243642e+00,4.902119e-01,7.791620,7.778796,1.150221e-01,-1.287245e-02,1.178814e-02,...,4.902119e-01,-3.733854e+00,3.243642e+00,-0.012754,1.178814e-02,0.000966,2.130197e-05,76.190205,20.0,0.4004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.048846,7.622054e-15,101.000000,-7.548040e-15,6.555609e-11,99.758851,-0.241149,-6.934987e-09,-4.282562e-10,6.617416e-09,...,6.555609e-11,-6.556353e-11,-7.548040e-15,-0.096152,6.617416e-09,0.096152,1.999038e+02,-21.000000,20.0,99.5996
996,0.048360,7.848434e-15,101.000000,-7.772221e-15,6.943178e-11,99.759337,-0.240663,-7.345008e-09,-4.535761e-10,7.008661e-09,...,6.943178e-11,-6.943900e-11,-7.772221e-15,-0.096152,7.008661e-09,0.096152,1.999038e+02,-21.000000,20.0,99.6997
997,0.047878,7.947975e-15,101.000000,-7.870795e-15,7.229086e-11,99.759819,-0.240181,-7.647485e-09,-4.722548e-10,7.297288e-09,...,7.229086e-11,-7.229798e-11,-7.870795e-15,-0.096152,7.297288e-09,0.096152,1.999038e+02,-21.000000,20.0,99.7998
998,0.047401,7.953532e-15,101.000000,-7.876299e-15,7.425796e-11,99.760296,-0.239704,-7.855600e-09,-4.851063e-10,7.495873e-09,...,7.425796e-11,-7.426506e-11,-7.876299e-15,-0.096152,7.495873e-09,0.096152,1.999038e+02,-21.000000,20.0,99.8999


## Try modeling the 1 step model with massaction

$$\text{Fuel + Substrate + Enzyme} \leftrightarrow^1 \text{Fuel:Substrate:Enzyme} \rightarrow^2 \text{Waste:Product:Enzyme} \leftrightarrow^3 \text{Waste + Product + Enzyme}$$

**Combined** <br>
$$\text{2 ATP + Glucose + Enzyme1} \leftrightarrow^1 \text{2ATP:Glucose:Enzyme1} \rightarrow^2 \text{2ADP:F16P:Enzyme1} \leftrightarrow^3 \text{2 ADP + F16P + Enzyme1}$$ <br>

**Non-ATP Generating Pathway** <br>
$$\text{F16P + 2 ADP + Enzyme2} \leftrightarrow^4 \text{2ADP:F16P:Enzyme2} \rightarrow^5 \text{2 ATP:Isobutanol:Enzyme2} \leftrightarrow^6 \text{2 ATP + Isobutanol + Enzyme2}$$ <br>

**ATP Generating Pathway** <br>
$$\text{3 ADP + Pi + F16P + Enzyme3} \leftrightarrow^7 \text{3ADPPi:F16P:Enzyme3} \rightarrow^8 \text{3ATP:Isobutanol:Enzyme3} \leftrightarrow^9 \text{3 ATP + Isobutanol + Enzyme3}$$ <br>

**ATP Degradation** <br>
$$\text{ATP} \rightarrow \text{ADP + Pi}$$


In [26]:
species = ['atp', 'glucose', 'e1', '2atp:glucose:e1', '2adp:f16p:e1', 'adp', 'pi', 'f16p',
           'e2', '2adp:f16p:e2','2atp:isobutanol:e2', 
           '3adppi:f16p:e3', 'e3' , '3atp:isobutanol:e3', 'isobutanol']


parameters = [('kf', 10), ('kr', 0.1), ('delta', 10)]



# reaction 1
rxn1_f = (['atp', 'atp', 'glucose', 'e1'], ['2atp:glucose:e1'],
           'massaction',  {'k': 'kf'})
                            
rxn1_r = ( ['2atp:glucose:e1'],['atp', 'atp', 'glucose', 'e1'], 
            'massaction', {'k': 'kr'})

# reaction 2
rxn2_f = ( ['2atp:glucose:e1'],['2adp:f16p:e1'], 
           'massaction', {'k': 'kf'})

# reaction 3
rxn3_f =  ( ['2adp:f16p:e1'], ['adp', 'adp', 'f16p', 'e1'], 
           'massaction', {'k': 'kf'})
 
rxn3_r =  (['adp', 'adp', 'f16p', 'e1'], ['2adp:f16p:e1'],
           'massaction', {'k': 'kr'})

# reaction 4
rxn4_f =  (['f16p', 'adp', 'adp', 'e2'], ['2adp:f16p:e2'],
           'massaction', {'k': 'kf'})

rxn4_r =  ( ['2adp:f16p:e2'], ['f16p', 'adp', 'adp', 'e2'],
           'massaction', {'k': 'kr'})

# reaction 5 
rxn5_f =  (['2adp:f16p:e2'], ['2atp:isobutanol:e2'],
           'massaction', {'k': 'kf'})


# reaction 6
rxn6_f =  (['2atp:isobutanol:e2'], ['atp', 'atp', 'isobutanol', 'e2'],
           'massaction', {'k': 'kf'})

rxn6_r =  (['atp', 'atp', 'isobutanol', 'e2'], ['2atp:isobutanol:e2'],
           'massaction', {'k': 'kr'})
    

# reaction 7
rxn7_f = ( ['adp', 'adp', 'adp', 'pi', 'f16p', 'e3'],['3adppi:f16p:e3'], 
              'massaction', {'k': 'kf'})

rxn7_r = ( ['3adppi:f16p:e3'],['adp', 'adp', 'adp', 'pi', 'f16p', 'e3'], 
              'massaction', {'k': 'kr'})

# reaction 8
rxn8_f = ( ['3adppi:f16p:e3'],['3atp:isobutanol:e3'], 
              'massaction', {'k': 'kf'})

# reaction 9
rxn9_f = ( ['3atp:isobutanol:e3'],['atp', 'atp','atp',  'isobutanol', 'e3'], 
              'massaction', {'k': 'kf'})

rxn9_r = ( ['atp', 'atp', 'atp', 'isobutanol', 'e3'],['3atp:isobutanol:e3'], 
              'massaction', {'k': 'kr'})

# ATP DEGRADATION
rxn_d_atp = (["atp"], ['adp', 'pi'], "massaction", {"k":"delta"})


reactions = [rxn1_f, rxn1_r, rxn2_f, rxn3_f, rxn3_r,rxn4_f,
             rxn4_r, rxn5_f, rxn6_f, rxn7_f, rxn6_r,
             rxn7_r, rxn8_f, rxn9_f, rxn9_r, rxn_d_atp]


# Initial Conditions             
x0 = {
    "glucose":100,
    "atp":200, 
    'e1': 200, 
    'e2':200,
    'e3':200
}


M = Model(species = species, reactions = reactions, parameters = parameters, initial_condition_dict = x0)
timepoints = np.linspace(0,10,100)
df_results = py_simulate_model(timepoints, Model = M, stochastic = False)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:85: UserWarning: The following species are uninitialized and their value has been defaulted to 0: 2atp:glucose:e1, 2adp:f16p:e1, adp, pi, f16p, 2adp:f16p:e2, 2atp:isobutanol:e2, 3adppi:f16p:e3, 3atp:isobutanol:e3, isobutanol, 
/anaconda3/lib/python3.7/site-packages/scipy/integrate/odepack.py:248: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)
odeint failed with mxstep=500...

In [27]:
# First plot glucose and isobutanol
p = bokeh.plotting.figure(width = 300, height = 250, 
                         x_axis_label = 'time',
                         y_axis_label = 'concentration')
p.line(timepoints, df_results['glucose'], color = 'green', legend_label = 'glucose')
p.line(timepoints, df_results['isobutanol'], color = 'orange', legend_label = 'isobutanol')
p.line(timepoints, df_results['f16p'], color = 'purple', legend_label = 'f16p')


p.legend.location = 'center_right'

# Plot ATP and ADP, NADPH, NADP+
p2 = bokeh.plotting.figure(width = 300, height = 250,
                           x_axis_label = 'time',
                         y_axis_label = 'concentration')
p2.line(timepoints, df_results['atp'], color = 'red', legend_label = 'atp')
p2.line(timepoints, df_results['pi'], color = 'blue', legend_label = 'pi')
p2.line(timepoints, df_results['adp'], color = 'green', legend_label = 'adp')
p2.legend.location = 'center_right'

#colors = ['#66c2a5', '#fc8d62','#8da0cb','#e78ac3']
colors = ['#e41a1c','#377eb8','#4daf4a','#984ea3']
p3 = bokeh.plotting.figure(width = 300, height = 250,
                           x_axis_label = 'time',
                         y_axis_label = 'concentration')
p3.line(timepoints, df_results['e1'], color = 'blue', legend_label = 'e1')
p3.line(timepoints, df_results['e2'], color = 'green', legend_label = 'e2')

p3.legend.location = 'center_right'


# Show plots
bokeh.io.show(p)
bokeh.io.show(p2)

In [28]:
df_results

,atp,glucose,e1,2atp:glucose:e1,2adp:f16p:e1,adp,pi,f16p,e2,2adp:f16p:e2,2atp:isobutanol:e2,3adppi:f16p:e3,e3,3atp:isobutanol:e3,isobutanol,time
0,200.000000,1.000000e+02,200.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,200.000000,0.000000e+00,0.000000,0.000000e+00,200.000000,0.000000,0.000000,0.00000
1,1.200009,2.435715e-03,126.660954,3.644655e+01,3.689249e+01,0.651743,0.029082,3.113303e-01,175.589070,1.799326e+01,6.417672,3.507362e-01,199.117235,0.532029,1.053493,0.10101
2,1.568660,5.322509e-04,159.705266,1.327418e+01,2.702055e+01,0.603942,0.048171,2.778854e-01,152.122132,2.586864e+01,22.009228,1.077443e+00,192.839269,6.083288,4.388249,0.20202
3,1.433452,1.877587e-04,180.307122,4.834417e+00,1.485846e+01,0.474366,0.081913,1.962264e-01,146.161103,2.087916e+01,32.959742,1.319789e+00,182.990456,15.689755,9.262267,0.30303
4,1.312642,7.050603e-05,190.977853,1.760677e+00,7.261470e+00,0.383212,0.165938,1.086371e-01,149.144346,1.317838e+01,37.677277,1.287863e+00,173.817152,24.894985,13.830643,0.40404
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.389868,-2.783038e-30,200.000000,1.867629e-27,-3.203384e-11,56.454593,56.454593,-5.006383e-13,171.368892,-2.584841e-09,28.631108,2.632874e-09,171.368892,28.631108,42.737785,9.59596
96,0.387221,4.873116e-30,200.000000,-1.230150e-27,-4.603278e-13,56.847060,56.847060,1.068901e-12,171.446856,-2.408678e-11,28.553144,7.889647e-12,171.446856,28.553144,42.893712,9.69697
97,0.384612,-9.221506e-31,200.000000,-1.661653e-27,7.542383e-12,57.236871,57.236871,-9.664933e-13,171.524297,6.129156e-10,28.475703,-6.286342e-10,171.524297,28.475703,43.048593,9.79798
98,0.382040,-2.210032e-30,200.000000,-5.335227e-28,2.550595e-11,57.624066,57.624066,1.393229e-12,171.601221,2.038847e-09,28.398779,-2.053168e-09,171.601221,28.398779,43.202442,9.89899


**No ATP regeneration?**

## Try modeling the three-step model version 1


$$\text{Substrate + Enzyme} \leftrightarrow^1 \text{Substrate:Enzyme}$$ <br>
$$\text{Fuel + Substrate:Enzyme} \leftrightarrow^2 \text{Fuel:Substrate:Enzyme} \rightarrow^3 \text{Waste:Product:Enzyme} \leftrightarrow^4 \text{Waste + Product:Enzyme}$$ <br>
$$\text{Product:Enzyme} \leftrightarrow^5 \text{Product + Enzyme}$$ <br>

**Common**
$$\text{Glucose + Enzyme1} \leftrightarrow^1 \text{Glucose:Enzyme1}$$ <br>
$$\text{2 ATP + Glucose:Enzyme1} \leftrightarrow^2 \text{2ATP:Glucose:Enzyme1} \rightarrow^3 \text{2adp:f16p:Enzyme1} \leftrightarrow^4 \text{2ADP + F16P:Enzyme1}$$ <br>
$$\text{F16P:Enzyme1} \leftrightarrow^5 \text{F16P + Enzyme1}$$ <br>

**Non-ATP Generating Pathway**
$$\text{F16P + Enzyme2} \leftrightarrow^6 \text{F16P:Enzyme2}$$ <br>
$$\text{2 ADP + F16P:Enzyme2} \leftrightarrow^7 \text{2adp:F16P:Enzyme2} \rightarrow^8 \text{2ATP:Isobutanol:Enzyme2} \leftrightarrow^9 \text{2 ATP + Isobutanol:Enzyme2}$$ <br>
$$\text{Isobutanol:Enzyme2} \leftrightarrow^{10} \text{Isobutanol + Enzyme2}$$ <br>


**ATP Generating Pathway**
$$\text{F16P + Enzyme3} \leftrightarrow^{11} \text{F16P:Enzyme3}$$ <br>
$$\text{3 ADP + Pi + F16P:Enzyme3} \leftrightarrow^{12} \text{3adppi:F16P:Enzyme3} \rightarrow^{13} \text{3ATP:Isobutanol:Enzyme3} \leftrightarrow^{14} \text{3 ATP + Isobutanol:Enzyme3}$$ <br>
$$\text{Isobutanol:Enzyme3} \leftrightarrow^{15} \text{Isobutanol + Enzyme3}$$ <br>


**ATP Degradation** <br>
$$\text{ATP} \rightarrow \text{ADP + Pi}$$

In [31]:
species = ['glucose', 'e1', 'atp', 'glucose:e1', '2atp:glucose:e1', '2adp:f16p:e1', 'adp', 'pi', 'f16p:e1', 'f16p',
          'e2', 'f16p:e2', '2adp:f16p:e2', '2atp:isobutanol:e2', 'isobutanol:e2',
          'e3', 'f16p:e3', '3adppi:f16p:e3', '3atp:isobutanol:e3', 'isobutanol:e3', 'isobutanol']

parameters = [('k_f', 10), ('k_r', 0.1), ('delta', 10)]



rxn1_f = (['glucose', 'e1'], ['glucose:e1'], 'massaction', {'k': 'k_f'})
rxn1_r = (['glucose:e1'], ['glucose', 'e1'], 'massaction', {'k': 'k_r'})

rxn2_f = (['atp', 'atp', 'glucose:e1'], ['2atp:glucose:e1'], 'massaction', {'k': 'k_f'})
rxn2_r = (['2atp:glucose:e1'], ['atp', 'atp', 'glucose:e1'], 'massaction', {'k': 'k_r'})

rxn3_f = (['2atp:glucose:e1'], ['2adp:f16p:e1'], 'massaction', {'k': 'k_f'})

rxn4_f = (['2adp:f16p:e1'], ['adp', 'adp', 'f16p:e1'], 'massaction', {'k': 'k_f'})
rxn4_r = (['adp', 'adp', 'f16p:e1'], ['2adp2pi:f16p:e1'], 'massaction', {'k': 'k_r'})

rxn5_f = (['f16p:e1'], ['f16p', 'e1'], 'massaction', {'k': 'k_f'})

rxn6_f = (['f16p', 'e2'], ['f16p:e2'], 'massaction', {'k': 'k_f'})
rxn6_r = (['f16p:e2'], ['f16p', 'e2'], 'massaction', {'k': 'k_r'})

rxn7_f = (['adp', 'adp','f16p:e2'], ['2adp:f16p:e2'], 'massaction', {'k': 'k_f'})
rxn7_r = (['2adp2pi:f16p:e2'], ['adp', 'adp', 'pi', 'pi', 'f16p:e2'], 'massaction', {'k': 'k_r'})

rxn8_f = (['2adp:f16p:e2'], ['2atp:isobutanol:e2'], 'massaction', {'k': 'k_f'})

rxn9_f = (['2atp:isobutanol:e2'], ['atp', 'atp', 'isobutanol:e2'], 'massaction', {'k': 'k_f'})
rxn9_r = (['atp', 'atp', 'isobutanol:e2'], ['2atp:isobutanol:e2'], 'massaction', {'k': 'k_r'})

rxn10_f = (['isobutanol:e2'], ['isobutanol', 'e2'], 'massaction', {'k': 'k_f'})
rxn10_r = (['isobutanol', 'e2'], ['isobutanol:e2'], 'massaction', {'k': 'k_r'})

rxn11_f = (['f16p', 'e3'], ['f16p:e3'], 'massaction', {'k': 'k_f'})
rxn11_r = ( ['f16p:e3'], ['f16p', 'e3'], 'massaction', {'k': 'k_r'})

rxn12_f = (['adp', 'adp', 'adp', 'pi', 'f16p:e3'], ['3adppi:f16p:e3'], 'massaction', {'k': 'k_f'})
rxn12_r = (['3adppi:f16p:e3'], ['adp', 'adp', 'adp', 'pi', 'f16p:e3'], 'massaction', {'k': 'k_r'})

rxn13_f = (['3adppi:f16p:e3'], ['3atp:isobutanol:e3'], 'massaction', {'k': 'k_f'})

rxn14_f = (['3atp:isobutanol:e3'], ['atp', 'atp', 'atp', 'isobutanol:e3'], 'massaction', {'k': 'k_f'})
rxn14_r = ( ['atp', 'atp', 'atp', 'isobutanol:e3'], ['3atp:isobutanol:e3'], 'massaction', {'k': 'k_r'})

rxn15_f = (['isobutanol:e3'], ['isobutanol', 'e3'], 'massaction', {'k': 'k_f'})
rxn15_r = (['isobutanol', 'e3'], ['isobutanol:e3'], 'massaction', {'k': 'k_r'})

rxn_d_atp = (['atp'], ['adp', 'pi'], 'massaction', {'k':'delta'})

reactions = [rxn1_f, rxn1_r, rxn2_f, rxn2_r, rxn3_f, rxn4_f, rxn4_r, rxn5_f, rxn6_f, rxn6_r, rxn7_f, rxn7_r,
            rxn8_f, rxn9_f, rxn9_r, rxn10_f, rxn10_r, rxn11_f, rxn11_r, rxn12_f, rxn12_r, rxn13_f,
            rxn14_f, rxn14_r, rxn15_f, rxn15_r, rxn_d_atp]

x_0 = {
    'glucose': 100,
    'atp':200,
    'e1':200,
    'e2':200,
    'e3':200
}

M = Model(species = species, reactions = reactions, parameters = parameters,
         initial_condition_dict = x_0)
timepoints = np.linspace(0,10,1000)
df_results = py_simulate_model(timepoints, Model = M, stochastic = False)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:65: UserWarning: The following species are uninitialized and their value has been defaulted to 0: glucose:e1, 2atp:glucose:e1, 2adp:f16p:e1, adp, pi, f16p:e1, f16p, f16p:e2, 2adp:f16p:e2, 2atp:isobutanol:e2, isobutanol:e2, f16p:e3, 3adppi:f16p:e3, 3atp:isobutanol:e3, isobutanol:e3, isobutanol, 2adp2pi:f16p:e1, 2adp2pi:f16p:e2, 


In [36]:
# First plot glucose and isobutanol
p = bokeh.plotting.figure(width = 300, height = 250, 
                         x_axis_label = 'time',
                         y_axis_label = 'concentration')
p.line(timepoints, df_results['glucose'], color = 'green', legend_label = 'glucose')
p.line(timepoints, df_results['isobutanol'], color = 'orange', legend_label = 'isobutanol')
p.line(timepoints, df_results['f16p'], color = 'purple', legend_label = 'f16p')


p.legend.location = 'center_right'

# Plot ATP and ADP, NADPH, NADP+
p2 = bokeh.plotting.figure(width = 300, height = 250,
                           x_axis_label = 'time',
                         y_axis_label = 'concentration')
p2.line(timepoints, df_results['atp'], color = 'red', legend_label = 'atp')
p2.line(timepoints, df_results['pi'], color = 'blue', legend_label = 'pi')
p2.line(timepoints, df_results['adp'], color = 'green', legend_label = 'adp')
p2.legend.location = 'center_right'

colors = ['#e41a1c','#377eb8','#4daf4a','#984ea3']
p3 = bokeh.plotting.figure(width = 300, height = 250,
                           x_axis_label = 'time',
                         y_axis_label = 'concentration')
p3.line(timepoints, df_results['e1'], color = 'blue', legend_label = 'e1')
p3.line(timepoints, df_results['e2'], color = 'green', legend_label = 'e2')

p3.legend.location = 'center_right'


# Show plots
bokeh.io.show(p)
bokeh.io.show(p2)
#bokeh.io.show(p3)

In [34]:
df_results

,glucose,e1,atp,glucose:e1,2atp:glucose:e1,2adp:f16p:e1,adp,pi,f16p:e1,f16p,...,isobutanol:e2,e3,f16p:e3,3adppi:f16p:e3,3atp:isobutanol:e3,isobutanol:e3,isobutanol,2adp2pi:f16p:e1,2adp2pi:f16p:e2,time
0,1.000000e+02,200.000000,2.000000e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,...,0.000000e+00,200.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.0,0.00000
1,2.957599e-03,100.012329,8.663152e+00,6.390083e+00,8.614311e+01,7.144369e+00,4.756530,4.121666,3.097134e-01,7.329636e-04,...,9.086991e-08,199.995681,3.054679e-03,1.243653e-03,2.084275e-05,2.934461e-07,4.547589e-09,0.000396,0.0,0.01001
2,3.996877e-04,100.086109,3.100520e+00,3.870960e+00,8.030648e+01,1.441268e+01,7.352559,4.616235,1.318580e+00,3.217781e-03,...,9.932542e-06,199.958752,1.517419e-02,2.509815e-02,9.466669e-04,2.934525e-05,9.609759e-07,0.005190,0.0,0.02002
3,3.219247e-04,100.290942,1.508088e+00,3.183136e+00,7.330617e+01,2.035019e+01,10.663511,4.744335,2.845591e+00,7.014407e-03,...,1.395499e-04,199.858190,2.599515e-02,1.087525e-01,6.726507e-03,3.357030e-04,1.754482e-05,0.023966,0.0,0.03003
4,2.908928e-04,100.666737,8.490030e-01,2.907623e+00,6.658397e+01,2.506534e+01,14.441538,4.679875,4.704700e+00,1.166165e-02,...,8.476636e-04,199.672638,3.350340e-02,2.686421e-01,2.356099e-02,1.655899e-03,1.226069e-04,0.071632,0.0,0.04004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1.091163e-12,170.405088,2.827420e-10,1.859396e-08,7.735946e-18,1.164924e-17,140.810175,140.810175,5.627170e-18,1.622095e-20,...,2.729900e+01,172.701000,1.437650e-22,4.934633e-18,1.151055e-10,2.729900e+01,1.580709e+01,29.594912,0.0,9.95996
996,1.091163e-12,170.405088,2.925430e-10,1.859396e-08,8.002880e-18,1.203418e-17,140.810175,140.810175,5.809414e-18,1.674894e-20,...,2.729900e+01,172.701000,1.483968e-22,4.999813e-18,1.190956e-10,2.729900e+01,1.580709e+01,29.594912,0.0,9.96997
997,1.091163e-12,170.405088,3.012247e-10,1.859396e-08,8.239241e-18,1.237393e-17,140.810175,140.810175,5.970017e-18,1.721508e-20,...,2.729900e+01,172.701000,1.524827e-22,5.050409e-18,1.226299e-10,2.729900e+01,1.580709e+01,29.594912,0.0,9.97998
998,1.091163e-12,170.405088,3.087060e-10,1.859396e-08,8.442820e-18,1.266529e-17,140.810175,140.810175,6.107465e-18,1.761506e-20,...,2.729900e+01,172.701000,1.559847e-22,5.085797e-18,1.256756e-10,2.729900e+01,1.580709e+01,29.594912,0.0,9.98999


**1 ATP Regeneration, why is isobutanol level so low, not enough phosphate for reaction**

## Try modeling three-step model version 2

$$\text{Enzyme + Fuel} \leftrightarrow^1 \text{Enzyme:Fuel}$$ <br>
$$\text{Substrate + Enzyme:Fuel} \leftrightarrow^2 \text{Fuel:Substrate:Enzyme} \rightarrow^3 \text{Waste:Product:Enzyme} \leftrightarrow^4 \text{Waste:Enzyme + Product}$$ <br>
$$\text{Waste:Enzyme}\leftrightarrow^5 \text{Waste + Enzyme}$$ <br>

**Combined**
$$\text{Enzyme1 + 2 ATP} \leftrightarrow^1 \text{Enzyme1:2ATP}$$ <br>
$$\text{Glucose + Enzyme1:2ATP} \leftrightarrow^2 \text{2ATP:Glucose:Enzyme1} \rightarrow^3 \text{2ADP:F16P:Enzyme1} \leftrightarrow^4 \text{2ADP:Enzyme1 + F16P}$$ <br>
$$\text{2ADP:Enzyme1}\leftrightarrow^5 \text{2 ADP + Enzyme1}$$ <br>

**Non-ATP Generating Pathway**
$$\text{Enzyme2 + 2 ADP} \leftrightarrow^6 \text{Enzyme2:2ADP}$$ <br>
$$\text{F16P + Enzyme2:2ADP} \leftrightarrow^7 \text{2ADP:F16P:Enzyme2} \rightarrow^8 \text{2ATP:Isobutanol:Enzyme2} \leftrightarrow^9 \text{2ATP:Enzyme2 + Isobutanol}$$ <br>
$$\text{2ATP:Enzyme2}\leftrightarrow^{10} \text{2 ATP + Enzyme2}$$ <br>

**ATP Generating Pathway**
$$\text{Enzyme3 + 3 ADP + Pi} \leftrightarrow^{11} \text{Enzyme3:3ADPPi}$$ <br>
$$\text{F16P + Enzyme3:3ADPPi} \leftrightarrow^{12} \text{3ADPPi:F16P:Enzyme3} \rightarrow^{13} \text{3ATP:F16P:Enzyme3} \leftrightarrow^{14} \text{3ATP:Enzyme3 + F16P}$$ <br>
$$\text{3ATP:Enzyme3}\leftrightarrow^{15} \text{3 ATP + Enzyme3}$$ <br>



In [47]:
species = ['e1', 'atp', 'e1:2atp', '2atp:glucose:e1', '2adp:f16p:e1', '2adp:e1', 'f16p', '2adp:e1', 'adp', 'pi',
          'e2', 'e2:2adp', '2adp:f16p:e2', '2atp:isobutanol:e2', '2atp:e2', 'isobutanol',
          'e3', 'e2:3adppi', '3adppi:f16p:e3', '3atp:f16p:e3', '3atp:e3']

parameters = [('kf', 10), ('kr', 0.1), ('delta', 10)]

rxn1f = (['e1', 'atp', 'atp'], ['e1:2atp'], 'massaction', {'k': 'kf'})
rxn1r = (['e1:2atp'], ['e1', 'atp', 'atp'], 'massaction', {'k': 'kf'})

rxn2f = (['glucose', 'e1:2atp'], ['2atp:glucose:e1'], 'massaction', {'k': 'kf'})
rxn2r = (['2atp:glucose:e1'], ['glucose', 'e1:2atp'], 'massaction', {'k': 'kr'})

rxn3f = (['2atp:glucose:e1'], ['2adp:f16p:e1'], 'massaction', {'k': 'kf'})

rxn4f = (['2adp:f16p:e1'], ['2adp:e1', 'f16p'], 'massaction', {'k': 'kf'})
rxn4r = (['2adp:e1', 'f16p'], ['2adp:f16p:e1'], 'massaction', {'k': 'kr'})

rxn5f = (['2adp:e1'], ['adp', 'adp', 'e1'], 'massaction', {'k': 'kf'})
rxn5r = (['adp', 'adp', 'e1'], ['2adp:e1'], 'massaction', {'k': 'kr'})

rxn6f = (['e2', 'adp', 'adp'], ['e2:2adp'], 'massaction', {'k': 'kf'})
rxn6r = (['e2:2adp'], ['e2', 'adp', 'adp'], 'massaction', {'k': 'kf'})

rxn7f = (['f16p', 'e2:2adp'], ['2adp:f16p:e2'], 'massaction', {'k': 'kf'})
rxn7r = (['2adp:f16p:e2'], ['f16p', 'e2:2adp'], 'massaction', {'k': 'kf'})

rxn8f = (['2adp:f16p:e2'], ['2atp:isobutanol:e2'], 'massaction', {'k': 'kf'})

rxn9f = (['2atp:isobutanol:e2'], ['2atp:e2', 'isobutanol'], 'massaction', {'k': 'kf'})
rxn9r = (['2atp:e2', 'isobutanol'], ['2atp:isobutanol:e2'], 'massaction', {'k': 'kr'})

rxn10f = (['2atp:e2'], ['atp', 'atp', 'e2'], 'massaction', {'k': 'kf'})
rxn10r = (['atp', 'atp', 'e2'], ['2atp:e2'], 'massaction', {'k': 'kr'})

rxn11f = (['e3', 'adp', 'adp', 'adp', 'pi'], ['e3:3adppi'], 'massaction', {'k': 'kf'})
rxn11r = (['e3:3adppi'], ['e3', 'adp', 'adp', 'adp', 'pi'], 'massaction', {'k': 'kr'})

rxn12f = (['f16p', 'e3:3adppi'], ['3adppi:f16p:e3'], 'massaction', {'k': 'kf'})
rxn12r = (['3adppi:f16p:e3'], ['f16p', 'e3:3adppi'], 'massaction', {'k': 'kr'})

rxn13f = (['3adppi:f16p:e3'], ['3atp:f16p:e3'], 'massaction', {'k': 'kf'})

rxn14f = (['3atp:f16p:e3'], ['3atp:e3', 'f16p'], 'massaction', {'k': 'kf'})
rxn14r = (['3atp:e3', 'f16p'], ['3atp:f16p:e3'], 'massaction', {'k': 'kr'})

rxn15f = (['3atp:e3'], ['atp', 'atp', 'atp', 'e3'], 'massaction', {'k': 'kf'})
rxn15r = (['atp', 'atp', 'atp', 'e3'], ['3atp:e3'], 'massaction', {'k': 'kr'})

rxn_d_atp = (["atp"], ['adp', 'pi'], "massaction", {"k":"delta"})

reactions = [rxn1f, rxn1r, rxn2f, rxn2r, rxn3f, rxn4f, rxn4r, rxn5f, rxn5r, rxn6f, rxn6r, rxn7f, rxn7r,
            rxn8f, rxn9f, rxn9r, rxn10f, rxn10r, rxn11f, rxn11r, rxn12f, rxn12r, rxn13f, rxn14f, rxn14r,
            rxn15f, rxn15r, rxn_d_atp]

x_0 = {
    'glucose': 100,
    'atp':200,
    'e1':200,
    'e2':200,
    'e3':200
}

M = Model(species = species, reactions = reactions, parameters = parameters,
         initial_condition_dict = x_0)
timepoints = np.linspace(0,1000,1000)
df_results = py_simulate_model(timepoints, Model = M, stochastic = False)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:64: UserWarning: The following species are uninitialized and their value has been defaulted to 0: e1:2atp, 2atp:glucose:e1, 2adp:f16p:e1, 2adp:e1, f16p, adp, pi, e2:2adp, 2adp:f16p:e2, 2atp:isobutanol:e2, 2atp:e2, isobutanol, e2:3adppi, 3adppi:f16p:e3, 3atp:f16p:e3, 3atp:e3, e3:3adppi, 
odeint failed with mxstep=500...

In [48]:
# First plot glucose and isobutanol
p = bokeh.plotting.figure(width = 300, height = 250, 
                         x_axis_label = 'time',
                         y_axis_label = 'concentration')
p.line(timepoints, df_results['glucose'], color = 'green', legend_label = 'glucose')
p.line(timepoints, df_results['isobutanol'], color = 'orange', legend_label = 'isobutanol')
p.line(timepoints, df_results['f16p'], color = 'purple', legend_label = 'f16p')
p.legend.location = 'center_right'

# Plot ATP and ADP, NADPH, NADP+
p2 = bokeh.plotting.figure(width = 300, height = 250,
                           x_axis_label = 'time',
                         y_axis_label = 'concentration')
p2.line(timepoints, df_results['atp'], color = 'red', legend_label = 'atp')
p2.line(timepoints, df_results['pi'], color = 'blue', legend_label = 'pi')
p2.line(timepoints, df_results['adp'], color = 'green', legend_label = 'adp')
p2.legend.location = 'center_right'

#colors = ['#66c2a5', '#fc8d62','#8da0cb','#e78ac3']
colors = ['#e41a1c','#377eb8','#4daf4a','#984ea3']
p3 = bokeh.plotting.figure(width = 300, height = 250,
                           x_axis_label = 'time',
                         y_axis_label = 'concentration')
p3.line(timepoints, df_results['e1'], color = 'blue', legend_label = 'e1')
p3.line(timepoints, df_results['e2'], color = 'green', legend_label = 'e2')

p3.legend.location = 'center_right'


# Show plots
bokeh.io.show(p)
bokeh.io.show(p2)
#bokeh.io.show(p3)

In [49]:
df_results

,e1,atp,e1:2atp,2atp:glucose:e1,2adp:f16p:e1,2adp:e1,f16p,adp,pi,e2,...,2atp:e2,isobutanol,e3,e2:3adppi,3adppi:f16p:e3,3atp:f16p:e3,3atp:e3,glucose,e3:3adppi,time
0,200.000000,200.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,200.000000,...,0.000000,0.000000,200.000000,0.0,0.000000,0.000000,0.000000,1.000000e+02,0.000000,0.000000
1,120.945556,0.689657,78.487142,5.522967e-03,1.242241e-01,4.375548e-01,3.036916,3.889601e-03,0.524347,183.301189,...,6.197439,86.003716,197.400018,0.0,0.413329,0.450751,1.602789,7.133462e-07,0.133113,1.001001
2,108.030173,0.852205,91.969377,2.485029e-07,3.289037e-05,4.166120e-04,2.514183,2.334713e-04,5.845581,196.332137,...,1.806108,95.069857,197.375940,0.0,0.264254,0.311934,1.940903,2.731748e-11,0.106969,2.002002
3,107.807250,0.855177,92.192609,1.119152e-11,3.392640e-06,1.375271e-04,2.314865,1.208470e-04,11.660357,196.637590,...,1.699367,95.437609,197.321873,0.0,0.275412,0.320501,1.961825,1.227238e-15,0.120390,3.003003
4,107.803598,0.855224,92.196308,4.965359e-16,2.092648e-06,9.116246e-05,2.206087,8.165005e-05,17.434301,196.665740,...,1.691704,95.558330,197.305543,0.0,0.278772,0.322079,1.965880,5.444687e-20,0.127727,4.004004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,107.991060,0.852005,92.008940,-6.394121e-82,2.715490e-09,2.777807e-07,0.977452,2.571996e-07,5658.340044,196.701500,...,1.675924,96.812839,197.155628,0.0,0.283986,0.303181,1.963762,-7.167059e-86,0.293442,995.995996
996,107.991121,0.852004,92.008879,-2.344992e-82,2.712317e-09,2.775022e-07,0.977290,2.569416e-07,5664.025871,196.701501,...,1.675921,96.813004,197.155587,0.0,0.283985,0.303177,1.963760,-2.549637e-86,0.293491,996.996997
997,107.991181,0.852003,92.008818,-3.174278e-82,2.709150e-09,2.772242e-07,0.977128,2.566841e-07,5669.711691,196.701503,...,1.675919,96.813170,197.155545,0.0,0.283985,0.303174,1.963757,-3.421324e-86,0.293539,997.997998
998,107.991242,0.852002,92.008758,-3.949111e-82,2.705991e-09,2.769468e-07,0.976966,2.564271e-07,5675.397504,196.701504,...,1.675917,96.813335,197.155503,0.0,0.283985,0.303170,1.963754,-4.232321e-86,0.293587,998.998999


**Why is Pi still being created?**

# Trying a simplified model again

**Combined**
$$\text{2 ATP + Glucose} \leftrightarrow^{E1} \text{2 ADP + F16P}$$
**Non-ATP Generating**
$$\text{2 ADP + F16P} \leftrightarrow^{E2} \text{2 ATP + Isobutanol}$$
**ATP Generating**
$$\text{3 ADP + Pi + F16P} \leftrightarrow^{E3} \text{3 ATP + Isobutanol}$$
**Spontaneous Hydrolysis**
$$\text{ATP} \rightarrow \text{ADP + Pi}$$


In [ ]:
species = ['atp', 'glucose', 'adp', 'pi', 'f16p', 'isobutanol']
parameters = [('kf', 1), ('kr', 0.1), ('delta', 10)]

rxn1 = (['atp', 'atp', 'glucose'], ['adp', 'adp', 'f16p'], 'massaction', {'k': 'kf'})
rxn2 = (['adp', 'adp', 'f16p'], ['atp', 'atp', 'isobutanol'], 'massaction', {'k': 'kf'})
rxn3 = (['adp', 'adp', 'adp', 'pi', 'f16p'], ['atp', 'atp', 'atp', 'isobutanol'], 'massaction', {'k': 'kf'})

rxn_d_atp = (['atp'], ['adp', 'pi'], 'massaction', {'k': 'delta'})


reactions = [rxn1, rxn2, rxn3, rxn_d_atp]

x_0 = {
    'glucose': 500,
    'atp':200
}

M = Model(species = species, reactions = reactions, parameters = parameters,
         initial_condition_dict = x_0)
timepoints = np.linspace(0,1,1000)
df_results = py_simulate_model(timepoints, Model = M, stochastic = False)

In [ ]:
# First plot glucose and isobutanol
p = bokeh.plotting.figure(width = 300, height = 250, 
                         x_axis_label = 'time',
                         y_axis_label = 'concentration')
p.line(timepoints, df_results['glucose'], color = 'green', legend_label = 'glucose')
p.line(timepoints, df_results['isobutanol'], color = 'orange', legend_label = 'isobutanol')
p.legend.location = 'center_right'

# Plot ATP and ADP, NADPH, NADP+
p2 = bokeh.plotting.figure(width = 300, height = 250,
                           x_axis_label = 'time',
                         y_axis_label = 'concentration')
p2.line(timepoints, df_results['atp'], color = 'red', legend_label = 'atp')
p2.line(timepoints, df_results['pi'], color = 'blue', legend_label = 'pi')
p2.line(timepoints, df_results['adp'], color = 'green', legend_label = 'adp')
#p2.line(timepoints, df_results['atp_extra'], color = 'purple', legend_label = 'atp_extra')
p2.legend.location = 'center_right'


# Show plots
bokeh.io.show(p)
bokeh.io.show(p2)

### Limiting factor is pi

In [ ]:
df_results

### show that u get more atp with the rheostat pathway